### Initiate llama-index OCI client

In [1]:
from llama_index.llms.oci_genai import OCIGenAI

llm = OCIGenAI(
    model="cohere.command-r-plus", # Switch to different models for testing
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="ocid1.tenancy.oc1..aaaaaaaaumuuscymm6yb3wsbaicfx3mjhesghplvrvamvbypyehh5pgaasna",
    auth_type="SECURITY_TOKEN",
    auth_profile="BoatOc1", # Please update here with your own profile name
    additional_kwargs={"max_tokens": 300}
)

### Simple Chat

In [2]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="Tell me a story"),
]
response = llm.chat(messages)
print(response)

assistant: Arrrr, me heartie! Do ye want to hear a tale that'll make yer heart race and yer blood boil? 

Once upon a time, there was a fearsome pirate captain named Mad Morgan. She was known far and wide for her cunning and her skill with a cutlass. Her ship, the *Black Raven*, was a terror on the high seas, striking fear into the hearts of merchants and sailors alike. 

One day, Captain Morgan heard a tale that set her heart aflame. It was said that a legendary treasure, known as the *Golden Monkey*, was hidden on a remote island in the Caribbean. This treasure was said to be worth a king's ransom, and it was guarded by a host of deadly traps and curses. 

Captain Morgan was undeterred. She gathered her crew and set sail for the island, determined to claim the *Golden Monkey* for herself. As they drew closer to their destination, the seas grew rough and the sky turned an ominous shade of purple. It was clear that this quest would not be an easy one. 

The *Black Raven* dropped anchor

In [3]:
print(response.raw)

{'status': 200, 'headers': {'content-type': 'application/json', 'opc-request-id': 'DECC0712EDC1440D9A9932C1A15B51DC/F47B6E48B5F73694D62A0D5E934B2000/99924C41E77B807AED7DF5D4BEFE4CC0', 'content-encoding': 'gzip', 'content-length': '912'}, 'data': {
  "chat_response": {
    "api_format": "COHERE",
    "chat_history": [
      {
        "message": "You are a pirate with a colorful personality",
        "role": "SYSTEM"
      },
      {
        "message": "Tell me a story",
        "role": "USER"
      },
      {
        "message": "Arrrr, me heartie! Do ye want to hear a tale that'll make yer heart race and yer blood boil? \n\nOnce upon a time, there was a fearsome pirate captain named Mad Morgan. She was known far and wide for her cunning and her skill with a cutlass. Her ship, the *Black Raven*, was a terror on the high seas, striking fear into the hearts of merchants and sailors alike. \n\nOne day, Captain Morgan heard a tale that set her heart aflame. It was said that a legendary treas

### Chat Streaming

In [4]:
resp = llm.stream_chat(messages)
for r in resp:
    print(r.delta, end="")

Arrrr, me heartie! Do ye want to hear a tale that'll make yer heart race and yer blood boil? 

Once upon a time, there was a fearsome pirate captain named Redbeard. He sailed the seven seas with his loyal crew, searching for treasure and glory. One day, Redbeard heard a tale of a legendary island hidden far away in the uncharted waters of the Great Ocean. It was said that this island held a fortune in gold and jewels beyond imagination. 

Redbeard was determined to find this island and claim its riches for himself. He set sail at once, his heart full of adventure and his eyes gleaming with greed. The journey was long and treacherous, filled with storms and sea monsters, but Redbeard and his crew were undaunted. They battled their way through the perils of the sea, their spirits high and their swords at the ready. 

Finally, after months of sailing, they spotted the island looming on the horizon. It was a wild and rugged place, surrounded by treacherous reefs and towering cliffs. Redbea

### Chat Chaining (Chat Prompt with a Template)

In [5]:
# Set up a chat prompt with a template
prompt_text = "Tell me a joke about {topic}"
formatted_prompt = prompt_text.format(topic="dogs")

# Send the prompt to OCIGenAI
response = llm.chat([ChatMessage(role="user", content=formatted_prompt)])
print(response.message.content)

Why do dogs make terrible dance partners? 

Because they have two left feet!


### Chat with Documents 

* Note: supported in Cohere command models 

In [29]:
from llama_index.core.llms import ChatMessage, MessageRole

documents = [
    {
        "title": "Oracle",
        "snippet": "Oracle database services and products offer customers cost-optimized and high-performance versions of Oracle Database, the world's leading converged, multi-model database management system, as well as in-memory, NoSQL and MySQL databases. Oracle Autonomous Database, available on premises via Oracle Cloud@Customer or in the Oracle Cloud Infrastructure, enables customers to simplify relational database environments and reduce management workloads.",
        "website": "https://www.oracle.com/database"
    }
]

messages = [
    ChatMessage(role=MessageRole.SYSTEM, content="You are an AI assistant."),
    ChatMessage(role=MessageRole.USER, content="Tell me something about Oracle."),
    ChatMessage(role=MessageRole.ASSISTANT, content="Oracle is a leading provider of database solutions."),
]

response = llm.chat(messages=messages, documents=documents)
print(response)
print(response.message.additional_kwargs['citations'])

assistant: Oracle is a provider of database services and products. It offers customers high-performance and cost-optimized versions of Oracle Database, a converged, multi-model database management system. Oracle Autonomous Database is available on-premises via Oracle Cloud@Customer and in the Oracle Cloud Infrastructure.
[{
  "document_ids": [
    "doc_0"
  ],
  "end": 55,
  "start": 12,
  "text": "provider of database services and products."
}, {
  "document_ids": [
    "doc_0"
  ],
  "end": 120,
  "start": 76,
  "text": "high-performance and cost-optimized versions"
}, {
  "document_ids": [
    "doc_0"
  ],
  "end": 139,
  "start": 124,
  "text": "Oracle Database"
}, {
  "document_ids": [
    "doc_0"
  ],
  "end": 193,
  "start": 143,
  "text": "converged, multi-model database management system."
}, {
  "document_ids": [
    "doc_0"
  ],
  "end": 220,
  "start": 194,
  "text": "Oracle Autonomous Database"
}, {
  "document_ids": [
    "doc_0"
  ],
  "end": 271,
  "start": 234,
  "text

### Basic tool calling in llamaindex 

only Cohere supports tool calling for now

In [33]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.tools import FunctionTool

def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Addition function on two integers."""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)

response = llm.chat_with_tools(
    tools=[add_tool, multiply_tool],
    user_msg="What is 3 * 12? Also, what is 11 + 49?",
)

print(response)
tool_calls = response.message.additional_kwargs.get('tool_calls', [])
print(tool_calls)

assistant: I will use the multiply tool to calculate 3 * 12 and the add tool to calculate 11 + 49.
[{'toolUseId': 'd28117f35b374389a343d103f22feef4', 'name': 'multiply', 'input': '{"a": 12, "b": 3}'}, {'toolUseId': '45a73237c2f642d8bdf3921155731840', 'name': 'add', 'input': '{"a": 11, "b": 49}'}]


### Chat streaming with tools

In [34]:
# Use streaming with tools
gen = llm.stream_chat_with_tools(
    tools=[multiply_tool, add_tool],
    user_msg="What is 23 * 45?",
)

# Process the stream
try:
    for response in gen:
        # Print content as it arrives
        if response.message.content:
            print(f"Content: {response.message.content}")

        # Check for tool calls
        if "tool_calls" in response.message.additional_kwargs:
            tool_calls = response.message.additional_kwargs["tool_calls"]
            print(f"Tool Calls: {tool_calls}")
except Exception as e:
    print(f"Error in stream processing: {e}")

Content: I
Content: I will
Content: I will use
Content: I will use the
Content: I will use the multiply
Content: I will use the multiply tool
Content: I will use the multiply tool to
Content: I will use the multiply tool to find
Content: I will use the multiply tool to find the
Content: I will use the multiply tool to find the answer
Content: I will use the multiply tool to find the answer to
Content: I will use the multiply tool to find the answer to the
Content: I will use the multiply tool to find the answer to the user
Content: I will use the multiply tool to find the answer to the user's
Content: I will use the multiply tool to find the answer to the user's request
Content: I will use the multiply tool to find the answer to the user's request.
Content: I will use the multiply tool to find the answer to the user's request.I will use the multiply tool to find the answer to the user's request.
Tool Calls: [{'toolUseId': '5caab4e07bd14a47826aab681f53a8be', 'name': 'multiply', 'input':

### Invoking tools and passing tool outputs back to model

In [35]:
# Initial user query
query = "What is 3 * 12? Also, what is 11 + 49?"
user_message = ChatMessage(role=MessageRole.USER, content=query)

# Pass initial message to the LLM with tool calling enabled
response = llm.chat_with_tools(tools=[add_tool, multiply_tool], user_msg=user_message.content)
print(response.message.content)  # Initial assistant response about using tools

# Extract tool calls and invoke each tool based on the response
messages = [user_message, response.message]  # Start with user message and initial assistant response

for tool_call in response.message.additional_kwargs.get("tool_calls", []):
    # Select the tool based on the tool call name
    selected_tool = {"add": add_tool, "multiply": multiply_tool}[tool_call["name"].lower()]
    tool_input = eval(tool_call["input"])  # Convert input JSON string to dict
    tool_output = selected_tool(**tool_input)  # Invoke tool

    # Add the tool's output as a new message in the conversation
    tool_output_message = ChatMessage(
        role=MessageRole.TOOL,
        content=str(tool_output),  # Format tool output as string
        additional_kwargs={"tool_call_id": tool_call.get("toolUseId")}
    )
    messages.append(tool_output_message)

# Pass the messages back to the LLM for a final response using the tool outputs
final_response = llm.chat(messages)
print(final_response)

I will use the multiply and add tools to answer this question.
assistant: 3 * 12 = **36** and 11 + 49 = **60**.


### Few-shot tool prompting & basic agent-like tool looping

In [36]:
import json
# Define initial few-shot messages
messages = [
    ChatMessage(
        role=MessageRole.USER,
        content="What's the product of 317253 and 128472 plus four"
    ),
    ChatMessage(
        role=MessageRole.ASSISTANT,
        content="",
        additional_kwargs={
            "tool_calls": [{"name": "multiply", "input": {"x": 317253, "y": 128472}, "toolUseId": "1"}]
        }
    ),
    ChatMessage(
        role=MessageRole.TOOL,
        content="16505054784",
        additional_kwargs={"tool_call_id": "1"}
    ),
    ChatMessage(
        role=MessageRole.ASSISTANT,
        content="",
        additional_kwargs={
            "tool_calls": [{"name": "add", "input": {"x": 16505054784, "y": 4}, "toolUseId": "2"}]
        }
    ),
    ChatMessage(
        role=MessageRole.TOOL,
        content="16505054788",
        additional_kwargs={"tool_call_id": "2"}
    ),
    ChatMessage(
        role=MessageRole.ASSISTANT,
        content="The product of 317253 and 128472 plus four is 16505054788"
    ),
]

# Define the query
query = "Whats 119 times 7 plus 30. Don't do any math yourself, only use tools for math. Respect order of operations. when you have the answer output 'the answer is: <answer>'"
messages.append(ChatMessage(role=MessageRole.USER, content=query))

# Start agent-like tool looping
while True:
    ai_response = llm.chat_with_tools(messages=messages, tools=[multiply_tool, add_tool])
    print(ai_response.message.content)

    if "the answer is:" in ai_response.message.content.lower():
        break

    messages.append(ai_response.message)

    # Execute tools based on tool calls and append tool responses
    if ai_response.message.additional_kwargs.get("tool_calls"):
        for tool_call in ai_response.message.additional_kwargs["tool_calls"]:
            tool_name = tool_call["name"]
            tool_input = tool_call["input"]

            # If tool_input is a string, parse it as JSON
            if isinstance(tool_input, str):
                tool_input = json.loads(tool_input)

            # Select tool and invoke it
            if tool_name == "multiply":
                result = multiply(**tool_input)
            elif tool_name == "add":
                result = add(**tool_input)

            # Append the tool result as a tool message
            messages.append(
                ChatMessage(
                    role=MessageRole.TOOL,
                    content=str(result),
                    additional_kwargs={"tool_call_id": tool_call["toolUseId"]}
                )
            )


I will use the multiply tool to multiply 119 and 7, then use the add tool to add 30 to the result.
119 x 7 = 833. I will now add 30 to this number.
The answer is: 863


### Create a llama-index agent

In [37]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings

# Set the LLM in the global settings
# Settings.llm = llm

# # Define your tools
# def multiply(a: int, b: int) -> int:
#     """Multiply two integers and return the result."""
#     return a * b
# 
# def add(a: int, b: int) -> int:
#     """Add two integers and return the result."""
#     return a + b
# 
# # Create FunctionTool instances
# multiply_tool = FunctionTool.from_defaults(fn=multiply)
# add_tool = FunctionTool.from_defaults(fn=add)
# tools = [multiply_tool, add_tool]

# Create the ReActAgent
agent = ReActAgent.from_tools(
    tools=[multiply_tool, add_tool],
    llm=llm,
    verbose=True,
)

# Start the conversation
response = agent.chat(
    "Whats 119 times 7 plus 30. Don't do any math yourself, only use tools for math. Respect order of operations."
)
print(response)


> Running step 51daca26-1dc3-40a5-ba0d-ce9dbdee7074. Step input: Whats 119 times 7 plus 30. Don't do any math yourself, only use tools for math. Respect order of operations.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 119, 'b': 7}
Observation: 833
> Running step 36e440fa-27b6-4c95-b0e2-dc3a0e1d6d00. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The answer is **863**.
The answer is **863**.
